##Download and prepare datasets

In [30]:

import argparse
import os
import random
import requests, zipfile, sys
import tarfile


import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
import torchvision.utils as utils
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import time
from PIL import Image





#Download train.tar -file from miniImageNet datset
miniimagenet_path = "./mini-imagenet/train.tar
with tarfile.open(miniimagenet_path, "r") as tf:
    print("Opened tarfile")
    tf.extractall(path="./")
    print("All files extracted")

data_dir = "./train"
data = np.load(data_dir)


#download EuroSAT(RGB) dataset
eurosat_path = "./EuroSAT_RGB"
try: 
    contents = os.listdir(folder_path)
    print("Contents of the folder:", contents)
except FileNotFoundError:
    print(f"The folder at {folder_path} was not found.")


#split data into training, validation and testing sets


Opened tarfile
All files extracted


IsADirectoryError: [Errno 21] Is a directory: './train'

#

## Pretrain a model

In [3]:
#hyperparameters
# Image Size
image_size = 32

# Batch size during training
batch_size = 128
num_workers = 1

# Learning rate for optimizers
lr = 0.0002

# Number of training epochs
num_epochs = 30

#Dataset to tensor
dataset = torchvision.datasets.ImageFolder(
    root='./train.............',
    transform=transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
)
#Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

# Plot some training images
real_batch = next(iter(dataloader))
plt.figure(figsize=(5, 5))
plt.axis('off')
plt.title('The Training Dataset')
plt.imshow(np.transpose(utils.make_grid(real_batch[0].to(device)[:36], padding=2, normalize=True, nrow=6).cpu(),(1,2,0)))

#model selection

#training

#evaluation

##Fine-tuning with EuroSAT dataset

In [ ]:
#select data

#fine tune

#tesing

##Model comparison and optimization

##Evaluation on additional dataset (bonus tasks)